# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

mongoimport --db uk_food --collection establishments --file /home/mo/Documents/UNC/homeWork/nosql-challenge/Starter_Code/Resources/establishments.json/establishments.json --jsonArray


In [24]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd

In [25]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [26]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'classDB', 'config', 'gardenDB', 'local', 'met', 'uk_food']


In [27]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [28]:
# review the collections in our new database
print(db.list_collection_names())

['establishments']


In [29]:
# review a document in the establishments collection
pprint(db.establishments.find_one())

{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6487d678556c2a05d5334401'),
 'geocode': {'latitude': Decimal128('51.083812'),
             'longitude': Decimal128('1.195625')},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1043695',
            'rel

In [30]:
# assign the collection to a variable
establishments = db['establishments']


Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'uk_food'), 'establishments')

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [31]:
# Create a dictionary for the new restaurant data
new_restrunt_data = {
"BusinessName":"Penang Flavours",
"BusinessType":"Restaurant/Cafe/Canteen",
"BusinessTypeID":"",
"AddressLine1":"Penang Flavours",
"AddressLine2":"146A Plumstead Rd",
"AddressLine3":"London",
"AddressLine4":"",
"PostCode":"SE18 7DY",
"Phone":"",
"LocalAuthorityCode":"511",
"LocalAuthorityName":"Greenwich",
"LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
"LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
"scores":{
"Hygiene":"",
"Structural":"",
"ConfidenceInManagement":""
},
"SchemeType":"FHRS",
"geocode":{
"longitude":"0.08384000",
"latitude":"51.49014200"
},
"RightToReply":"",
"Distance":4623.9723280747176,
"NewRatingPending":True
} 


In [32]:
# Insert the new restaurant into the collection
data_query = db.establishments
data_query.insert_one(new_restrunt_data)

In [33]:
# Check that the new restaurant was inserted
check = data_query.find_one({
 "AddressLine2":"146A Plumstead Rd"})
pprint(check)


{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': '',
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6487d69aaf2b0b56de5d3f4e'),
 'geocode': {'latitude': Decimal128('51.49014200'),
             'longitude': Decimal128('0.08384000')},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [58]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {'BusinessType': 'Restaurant/Cafe/Canteen'}
fields = {'BusinessType': 1  , 'BusinessTypeID': 1 , '_id' : 0}

BusinessTypeID = data_query.find(query,fields)
pprint(BusinessTypeID)
for result in BusinessTypeID:
    pprint(result)


{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': ''}
{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}
{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}
{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}
{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}
{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}
{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}
{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}
{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}
{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}
{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}
{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}
{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}
{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}
{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}
{'BusinessType': 'Restau

3. Update the new restaurant with the `BusinessTypeID` you found.

In [64]:
# Update the new restaurant with the correct BusinessTypeID
query = {'AddressLine2': '146A Plumstead Rd'}
update = {'$set': {'BusinessTypeID': '1'}}
result = data_query.update_one(query,update)



In [65]:
# Confirm that the new restaurant was updated
data_query.find_one({
 "AddressLine2":"146A Plumstead Rd"})


{'_id': ObjectId('6487d69aaf2b0b56de5d3f4e'),
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': '1',
 'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'PostCode': 'SE18 7DY',
 'Phone': '',
 'LocalAuthorityCode': '511',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'scores': {'Hygiene': '', 'Structural': '', 'ConfidenceInManagement': ''},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': Decimal128('0.08384000'),
  'latitude': Decimal128('51.49014200')},
 'RightToReply': '',
 'Distance': 4623.972328074718,
 'NewRatingPending': True}

In [37]:
# we can also use result.modified_count
pprint(result.modified_count)

1


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [38]:
# Find how many documents have LocalAuthorityName as "Dover"
query = {"LocalAuthorityName" : "Dover"}
count_local = data_query.count_documents(query)
print(f" the number of documents tha has LocalAuthorityName as Dover is : {count_local}")

 the number of documents tha has LocalAuthorityName as Dover is : 0


In [39]:
# Delete all documents where LocalAuthorityName is "Dover"

result = data_query.delete_many(query)
result



In [40]:
# Check if any remaining documents include Dover
query = {"LocalAuthorityName" : "Dover"}
print(data_query.find_one(query))

None


In [41]:
# Check that other documents remain with 'find_one'
data_query.find_one()

{'_id': ObjectId('6487d678556c2a05d5334401'),
 'FHRSID': 1043695,
 'ChangesByServerID': 0,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': '',
 'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'PostCode': 'CT19 6BL',
 'Phone': '',
 'RatingValue': '5',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingDate': '2018-04-04T00:00:00',
 'LocalAuthorityCode': '188',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 5},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': Decimal128('1.195625'),
  'latitude': Decimal128('51.083812')},
 'RightToReply': '',
 'Distance': 4591.765489457773,
 'NewRatingPending': False,
 'meta': {'dataSource': None,
 

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

Use `update_many` to convert `RatingValue` to integer numbers.

In [43]:
establishments.update_many({}, [ {'$set':{ "geocode.longitude" : {'$toDecimal': "$geocode.longitude"},
"geocode.latitude" : {'$toDecimal': "$geocode.latitude"}
}} ]) 

In [48]:
#finding one document to find the type
document = establishments.find_one({'AddressLine2': '146A Plumstead Rd'})

# Get the value of a field (e.g., longitude)
longitude = document.get('geocode', {}).get('longitude')
print(type(longitude))

<class 'bson.decimal128.Decimal128'>


In [45]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [76]:
establishments.update_many({}, {'$set': {'RatingValue': {'$toInt': "$RatingValue"}}})


In [77]:
# Check that the coordinates and rating value are now numbers
#finding one document to find the type



check = establishments.find_one()
pprint((check))


{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': '',
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': {'$toInt': '$RatingValue'},
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6487d678556c2a05d5334401'),
 'geocode': {'latitude': Decimal128('51.083812'),
             'longitude': Decimal128('1.195625')},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/10